# Demonstration of xarray apply_ufunc  
Allows to apply functions to each point in specific dimension, e.g. perform a correlation at each lat,lon grid point or as shown below convert salinities along each profile. By using dask the tasks are parallelized and the computation is pretty quick. 

In [4]:
import xarray as xr
import numpy as np 
import dask
import gsw  # Gibbs Seawater library (have to install first likely)


# load dataset
woa = xr.open_dataset('./data/woa_example_data.nc')

In [14]:
# 1) Derive absolute salinity from practical salinity and conservative temperature from in-situ temperature
# output size does not change

##Absolute Salinity
# create function which is called by apply_ufunct
def SA_from_SP(psal,lon,lat):
    result = gsw.SA_from_SP(psal,woa.pres,lon,lat) #output format of function had to be right, which is why I applied the squeeze function
    return np.squeeze(result)


result = xr.apply_ufunc(SA_from_SP,woa.sal,woa.lon,woa.lat,
                        input_core_dims=[['pres'],[],[]],
                        output_core_dims=[["pres"]],
                        vectorize=True,
                        dask='parallelized',
                        output_dtypes=[np.dtype(float)],
                        dask_gufunc_kwargs={'output_sizes':{'pres':200}})#

woa['SA'] = result.T


## Consertvative Temperature
def CT_from_t(SA,t):
    result = gsw.CT_from_t(SA,t,woa.pres) #output format of function had to be right, which is why I applied the squeeze function
    return np.squeeze(result)

result = xr.apply_ufunc(CT_from_t,woa.SA,woa.temp,
                        input_core_dims=[['pres'],['pres']],
                        output_core_dims=[["pres"]],
                        vectorize=True,
                        dask='parallelized',
                        output_dtypes=[np.dtype(float)],
                        dask_gufunc_kwargs={'output_sizes':{'pres':200}})#
woa['CT'] = result.T

In [16]:
# 2) ### Buoyancy frequency
# the buoyancy frequency is derived based on the density gradient and
# thus the output is N-1, which is why we have to define a new output dimension

def bfrq(SA,CT,lat):
    result = gsw.Nsquared(SA,CT,woa.pres,lat) 
    return np.squeeze(result[0])

result = xr.apply_ufunc(bfrq,woa.SA,woa.CT,woa.lat,
                        input_core_dims=[['pres'],['pres'],[]],
                        output_core_dims=[["pres_new"]],
                        vectorize=True,
                        dask='parallelized',
                        output_dtypes=[np.dtype(float)],
                        dask_gufunc_kwargs={'output_sizes':{'pres_new':199}})#

# create xarray, interpolate to woa pres and add to woa dataset
dummy = result.rename({'pres_new':'pres'})
dummy["pres"] = np.arange(1.5,200.5)
woa['bfrq'] = dummy.interp(pres=woa.pres.values,kwargs={"fill_value": "extrapolate"}).T